In [91]:
import pandas as pd

In [92]:
df = pd.read_csv('labour.csv')

 - Here I am aiming to save my dataset in mysql database and use it in my project. I will use the sqlalchemy library for this.
 - I will have to define an engine() for this process, but before that, two different situations arise as a connection string. This is because I will first connect to my database and create a database with a name I want. Then, in order to save my dataset to the database, I need to reconnect my engine() connection with the name of the database I created.
 - It looks like a code mess, but whatever method I tried, I couldn't succeed. For the reasons I explained in the previous article, I had to manage engine() with two different connection strings in this way.
 - Instead of using a local mysql solution, I created an azure mysql database, which is one of the free services specially defined for students, using the azure portal of the microsoft company and I will use it in my project.

In [116]:
from sqlalchemy import create_engine, text

user = 'ca2023'
password = 'CCT2023!'
host = 'cctmysql.mysql.database.azure.com'
port = 3306
database = 'cct_db'

engine_url = f'mysql+mysqlconnector://{user}:{password}@{host}:{port}'
db_url = f'mysql+mysqlconnector://{user}:{password}@{host}:{port}/{database}'

engine = create_engine(engine_url, echo=True)

with engine.connect() as conn:
    conn.execute(text(f"CREATE DATABASE IF NOT EXISTS {database};"))
    conn.close()

    

2023-05-07 21:51:47,750 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2023-05-07 21:51:47,751 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-05-07 21:51:48,265 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2023-05-07 21:51:48,267 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-05-07 21:51:48,518 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2023-05-07 21:51:48,519 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-05-07 21:51:48,852 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-05-07 21:51:48,854 INFO sqlalchemy.engine.Engine CREATE DATABASE IF NOT EXISTS cct_db;
2023-05-07 21:51:48,855 INFO sqlalchemy.engine.Engine [generated in 0.16937s] {}
2023-05-07 21:51:48,937 INFO sqlalchemy.engine.Engine ROLLBACK


 - Now that I have created my database, I will send the dataframe to my database using my connection string containing the database for engine().
 - I carry out my process by checking for errors due to the possibility of the current table name being in the database.
 - I am performing my operation using the **to_sql()** function of the pandas library.

In [117]:
engine = create_engine(db_url, echo=True)
try:
    df.to_sql('labour', con=engine, if_exists='fail', index=False)
except Exception as ex:
    print(ex)
else:
    print("Table created")

2023-05-07 21:51:50,854 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2023-05-07 21:51:50,855 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-05-07 21:51:51,330 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2023-05-07 21:51:51,330 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-05-07 21:51:51,585 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2023-05-07 21:51:51,586 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-05-07 21:51:51,993 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-05-07 21:51:51,994 INFO sqlalchemy.engine.Engine DESCRIBE `cct_db`.`labour`
2023-05-07 21:51:51,995 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-05-07 21:51:52,076 INFO sqlalchemy.engine.Engine ROLLBACK
Table 'labour' already exists.


In [118]:
conn = engine.connect()
sqldf = pd.read_sql_table('labour', conn)

2023-05-07 21:51:53,822 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-05-07 21:51:53,822 INFO sqlalchemy.engine.Engine DESCRIBE `cct_db`.`labour`
2023-05-07 21:51:53,823 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-05-07 21:51:54,095 INFO sqlalchemy.engine.Engine SHOW CREATE TABLE `labour`
2023-05-07 21:51:54,096 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-05-07 21:51:54,356 INFO sqlalchemy.engine.Engine SELECT labour.`STRUCTURE`, labour.`STRUCTURE_ID`, labour.freq, labour.indic_bt, labour.nace_r2, labour.s_adj, labour.unit, labour.geo, labour.`TIME_PERIOD`, labour.`OBS_VALUE`, labour.`OBS_FLAG` 
FROM labour
2023-05-07 21:51:54,357 INFO sqlalchemy.engine.Engine [generated in 0.17773s] {}


In [121]:
sqldf.head()

,STRUCTURE,STRUCTURE_ID,freq,indic_bt,nace_r2,s_adj,unit,geo,TIME_PERIOD,OBS_VALUE,OBS_FLAG
0,dataflow,ESTAT:STS_COLB_A(1.0),A,EMPL,F,NSA,I10,AL,2005,76.9,None
1,dataflow,ESTAT:STS_COLB_A(1.0),A,EMPL,F,NSA,I10,AL,2006,76.7,None
2,dataflow,ESTAT:STS_COLB_A(1.0),A,EMPL,F,NSA,I10,AL,2007,74.1,None
3,dataflow,ESTAT:STS_COLB_A(1.0),A,EMPL,F,NSA,I10,AL,2008,97.3,None
4,dataflow,ESTAT:STS_COLB_A(1.0),A,EMPL,F,NSA,I10,AL,2009,103.7,None


In [122]:
sqldf.shape

(9314, 11)

In [123]:
sqldf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9314 entries, 0 to 9313
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   STRUCTURE     9314 non-null   object 
 1   STRUCTURE_ID  9314 non-null   object 
 2   freq          9314 non-null   object 
 3   indic_bt      9314 non-null   object 
 4   nace_r2       9314 non-null   object 
 5   s_adj         9314 non-null   object 
 6   unit          9314 non-null   object 
 7   geo           9314 non-null   object 
 8   TIME_PERIOD   9314 non-null   int64  
 9   OBS_VALUE     9108 non-null   float64
 10  OBS_FLAG      1717 non-null   object 
dtypes: float64(1), int64(1), object(9)
memory usage: 800.6+ KB
